In [36]:
import os
import re
import time
from dotenv import load_dotenv
from github import Github

load_dotenv()



token = os.getenv('GITHUB_TOKEN')
g = Github(token)

# Trabajar sobre el repositorio específico
repo = g.get_repo("AlejandroDiBattista/tup25-p4")

print(f"         Repositorio: {repo.name}")
print(f"         Descripción: {repo.description}")
print(f"           Estrellas: {repo.stargazers_count}")
print(f"               Forks: {repo.forks_count}")
print(f"  Lenguaje principal: {repo.language}")
print(f"Última actualización: {repo.updated_at}")

         Repositorio: tup25-p4
         Descripción: TUP 2025 - Programación 4
           Estrellas: 5
               Forks: 60
  Lenguaje principal: JavaScript
Última actualización: 2025-09-02 00:29:41+00:00


In [37]:
# Listar todos los pull requests pendientes (abiertos)

def normalizar_pr(tp=1):
    def normalizar_titulo_pr(titulo):
        if match:= re.search(r'(\d{5})[,\s-]+(.+)\s*', titulo):
            return f"TP{tp} - {match.group(1)} - {match.group(2).title()}"
        else:
            return titulo

    open_pulls = repo.get_pulls(state='open', sort='created', base='main')
    print(f"> Pull Requests pendientes:")
    for pr in open_pulls:
        pr = repo.get_pull(pr.number)  # Refrescar datos del PR
        nuevo = normalizar_titulo_pr(pr.title)
        if pr.title != nuevo:
            pr.edit(title=nuevo)
        print(f"- #{pr.number}: {nuevo} (por {pr.user.login}) {"*" if pr.title != nuevo else ""}")
    print(".")

normalizar_pr(tp=1)

> Pull Requests pendientes:
- #3: TP1 - 61627 - Juarez Fernandez, Lourdes (por LourdesFernandez1) 
- #5: TP1 - 61315 - Thiago Robles (por T1T0Dev) 
- #6: TP1 - 61911 - Luciano Llanos (por LucianoLlanos) 
- #7: TP1 - 61221 - Marcelo Ezequiel Duclós (por EzeDuclos) 
- #8: TP1 - 61579 - Marti Gonzalo (por Lived18) 
- #9: TP1 - 61312 - Lucas David Paz Berrondo (por Lucasdxvid) 
- #10: TP1 - 61033 - Quiroga José Maria (por jose-qui) 
- #11: TP1 - 62555 - Verón, Matias Gabriel (por MatiasVeron12503) 
- #12: TP1 - 61096 - Suarez Vece Leandro (por leandroVece) 
- #13: TP1 - 61032 - Moyamariano (por mariano-moya) 
- #14: TP1 - 61271 - Donelli-Gerardo (por GerardoExe) 
- #15: TP1 - 61719 - Amoedopablo (por Kryfter) 
- #16: TP1 - 62172 - Salcedomariana (por marianasalcedo22) 
- #17: TP1 - 61676 - Maximiliano, Massey (por EPILONgaga) 
- #18: TP1 - 61319 - Carabajal José Gabriel (por 7GabrielC) 
- #19: TP1 - 61139 - Gallo, María Matilde (por matildegallom) 
- #20: TP1 - 61037 - Paz, Rodrigo  (por R

In [40]:
# Función para hacer merge de PRs sin conflictos
def merge_prs_sin_conflictos():
    open_pulls = repo.get_pulls(state='open', sort='created', base='main')
    print("> Intentando merge de PRs sin conflictos:")
    for pr in open_pulls:
        pr = repo.get_pull(pr.number)  # Refrescar datos
        # Esperar hasta que mergeable no sea None
        intentos = 0
        max_intentos = 12  # Máximo 12 * 5 = 60 segundos
        while pr.mergeable is None and intentos < max_intentos:
            print(f"Esperando mergeability para PR #{pr.number}... ({intentos+1}/{max_intentos})")
            time.sleep(5)
            pr = repo.get_pull(pr.number)  # Refrescar
            intentos += 1

        if pr.mergeable is True:  # Solo si puede ser merged
            try:
                pr.merge()
                print(f"✓ PR #{pr.number} merged: {pr.title}")
            except Exception as e:
                print(f"✗ Error al mergear PR #{pr.number}: {e}")
        elif pr.mergeable is False:
            print(f"✗ PR #{pr.number} tiene conflictos: {pr.title}")
        else:
            print(f"✗ Tiempo agotado para PR #{pr.number}: {pr.title}")
    print(".")

# Ejecutar la función
merge_prs_sin_conflictos()

> Intentando merge de PRs sin conflictos:
Esperando mergeability para PR #17... (1/12)
✗ PR #17 tiene conflictos: TP1 - 61676 - Maximiliano, Massey
✗ PR #37 tiene conflictos: TP1 - 62175 - Quiroga Marcela Noemí
✗ PR #39 tiene conflictos: TP1 - 61565 - Carrizo Mauro Martín
✗ PR #43 tiene conflictos: TP1 - 62055 - Aiquen Ahumada
.
